In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import lightgbm as lgb

In [2]:
total = pd.read_csv('한국가스공사_시간별 공급량_20181231.csv', encoding='cp949')

In [3]:
total.head()

,연월일,시간,구분,공급량
0,2013-01-01,1,A,2497.129
1,2013-01-01,2,A,2363.265
2,2013-01-01,3,A,2258.505
3,2013-01-01,4,A,2243.969
4,2013-01-01,5,A,2344.105


# preprocessing

In [4]:
total['구분'].unique

<bound method Series.unique of 0         A
1         A
2         A
3         A
4         A
         ..
368083    H
368084    H
368085    H
368086    H
368087    H
Name: 구분, Length: 368088, dtype: object>

In [5]:
d_map = {}
for i, d in enumerate(total['구분'].unique()):
    d_map[d] = i
total['구분'] = total['구분'].map(d_map)
# total['구분']

In [6]:
total['연월일'] = pd.to_datetime(total['연월일'])
# total['연월일'] 

In [7]:
total['year'] = total['연월일'].dt.year
total['month'] = total['연월일'].dt.month
total['day'] = total['연월일'].dt.day
total['weekday'] = total['연월일'].dt.weekday

In [8]:
train_years = [2013,2014,2015,2016,2017]
val_years = [2018]

In [9]:
train = total[total['year'].isin(train_years)]
val = total[total['year'].isin(val_years)]

In [10]:
features = ['구분', 'month', 'day', 'weekday', '시간']
train_x = train[features]
train_y = train['공급량']

val_x = val[features]
val_y = val['공급량']

# training
###### light gbm 핵심 파라미터 소개 
- application : 가장 중요한 파라미터로, 모델의 어플리케이션을 정하는데 이것이 regression 회귀분석 문제인지 또는 classification 분류 문제인지를 정합니다. Light GBM에서 디폴트는 regression 회귀분석 모델
    - regression: 회귀분석
    - binary: 이진 분류
    - multiclass: 다중 분류
    
- boosting : 실행하고자 하는 알고리즘 타입을 정의합니다. 디폴트값은 gdbt 
    - gdbt : Traditional Gradient Boosting Decision Tree
    - rf : Random Forest
    - dart : Dropouts meet Multiple Additive Regression Trees
    - goss : Gradient-based One-Side Sampling
    
- num_boost_round : boosting iteration 수로 일반적으로 100 이상

- learning_rate : 최종 결과에 대한 각각의 Tree에 영향을 미치는 변수입니다. GBM은 초기의 추정값에서 시작하여 각각의Tree 결과를 사용하여 추정값을 업데이트 합니다. 학습 파라미터는 이러한 추정에서 발생하는 변화의 크기를 컨트롤합니다. 일반적인 값은 0.1, 0.001, 0.003 등등

- num_leaves : 전체 Tree의 leave 수 이고, 디폴트값은 31

- device : 디폴트 값은 cpu 인데 gpu로 변경

- metric : 모델을 구현할 때 손실을 정하기 때문에 중요한 변수 중에 하나입니다. regression과 classification 을 위한 일반적인 손실 값이 아래에 나와있습니다.
    - mae : mean absolute error
    - mse : mean squared error
    - binary_logloss : loss for binary classification
    - multi_logloss : loss for multi classification

- max_bin : feature 값의 최대 bin 수를 의미

- categorical_feature : 범주형 feature의 인덱스를 의미합니다. 만약 categorical_features 가 0, 1, 2 이면 column 0, column 1, column 2 가 범주형 변수들입니다.

- ignore_column : categorical_features와 동일한 것인데 범주형 feature로써 특정 칼럼을 고려하지 않는 것입니다. 그 변수들을 무시하는 것입니다.

- save_binary : 데이터 파일의 메모리 사이즈를 처리해야 한다면 이 파라미터 값을 True로 설정하십시오. 이 값을 True로 세팅하면 데이터 세트를 바이너리 파일로 저장할 것이고, 이 바이너리 파일은 다음에 데이터를 읽어올 때 그 속도를 줄여줄 것입니다.

In [11]:
d_train = lgb.Dataset(train_x, train_y)
d_val = lgb.Dataset(val_x, val_y)

# params = {
#     'learning_rate' : 0.001,
#     'objective': 'regression',
#     'metric':'mae',
#     'sub_feature' : 0.5,
#     'num_leaves': 40,
#     'min_data' : 50,
#     'max_depth' : 20,
#     'seed':42
# }
params = {
    'objective': 'regression',
    'metric':'mae',
    'seed':42
}

model = lgb.train(params, d_train, 10000, d_val, verbose_eval=30, early_stopping_rounds=300)

/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:240: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002536 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 84
[LightGBM] [Info] Number of data points in the train set: 306768, number of used features: 5
[LightGBM] [Info] Start training from score 934.864036
Training until validation scores don't improve for 300 rounds
[30]	valid_0's l1: 197.34
[60]	valid_0's l1: 158.739
[90]	valid_0's l1: 151.586
[120]	valid_0's l1: 150.463
[150]	valid_0's l1: 148.47
[180]	valid_0's l1: 147.586
[210]	valid_0's l1: 147.09
[240]	valid_0's l1: 146.976
[270]	valid_0's l1: 146.64
[300]	valid_0's l1: 146.199
[330]	valid_0's l1: 146.34
[360]	valid_0's l1: 146.45
[390]	valid_0's l1: 146.632
[420]	valid_0's l1: 146.303
[450]	valid_0's l1: 146.309
[480]	valid_0's l1: 146.713
[510]	valid_0's l1: 147.4
[540]	valid_0's l1: 147.864
[570]	valid_0's l1: 147.922
[600]	va

# inference

In [12]:
test = pd.read_csv('data/test.csv')
submission = pd.read_csv('data/sample_submission.csv')

In [13]:
test.head()

,일자|시간|구분
0,2019-01-01 01 A
1,2019-01-01 02 A
2,2019-01-01 03 A
3,2019-01-01 04 A
4,2019-01-01 05 A


In [14]:
submission.head()

,일자|시간|구분,공급량
0,2019-01-01 01 A,0
1,2019-01-01 02 A,0
2,2019-01-01 03 A,0
3,2019-01-01 04 A,0
4,2019-01-01 05 A,0


In [15]:
test['일자'] = test['일자|시간|구분'].str.split(' ').str[0]
test['시간'] = test['일자|시간|구분'].str.split(' ').str[1].astype(int)
test['구분'] = test['일자|시간|구분'].str.split(' ').str[2]

In [16]:
test['일자'] = pd.to_datetime(test['일자'])
test['year'] = test['일자'].dt.year
test['month'] = test['일자'].dt.month
test['day'] = test['일자'].dt.day
test['weekday'] = test['일자'].dt.weekday

In [17]:
test['구분'] = test['구분'].map(d_map)

In [18]:
test_x = test[features]

In [19]:
test_x

,구분,month,day,weekday,시간
0,0,1,1,1,1
1,0,1,1,1,2
2,0,1,1,1,3
3,0,1,1,1,4
4,0,1,1,1,5
...,...,...,...,...,...
15115,6,3,31,6,20
15116,6,3,31,6,21
15117,6,3,31,6,22
15118,6,3,31,6,23


In [20]:
preds = model.predict(test_x)

In [21]:
submission['공급량'] = preds

In [22]:
submission.to_csv('baseline.csv', index=False)

In [23]:
submission

,일자|시간|구분,공급량
0,2019-01-01 01 A,2280.015244
1,2019-01-01 02 A,2092.407079
2,2019-01-01 03 A,1986.195957
3,2019-01-01 04 A,2017.484892
4,2019-01-01 05 A,2163.995167
...,...,...
15115,2019-03-31 20 H,492.515309
15116,2019-03-31 21 H,489.040445
15117,2019-03-31 22 H,465.536673
15118,2019-03-31 23 H,381.883757
